# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
mars_soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
all_data = mars_soup.find_all('tr', class_='data-row')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
rows = []

# Loop through the scraped data to create a list of rows
for row_tag in all_data:
    columns = row_tag.find_all('td')
    row_data = []
    for column in columns:
        row_data.append(column.text)

    rows.append(row_data)


In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
header_row = mars_soup.find('tr')
column_names = []
for column in header_row.find_all('th'):
    column_names.append(column.text)
    
column_names

In [ ]:
df = pd.DataFrame(rows, columns=column_names)


In [ ]:
# Confirm DataFrame was created successfully
df

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
df_datatypes = df.dtypes
df_datatypes

In [ ]:
# Change data types for data analysis
convert_dict = {'id': int,
               'sol': int,
               'ls': int,
               'month': int,
               'min_temp': float,
               'pressure': float
              }

df["terrestrial_date"]= pd.to_datetime(df["terrestrial_date"])
df = df.astype(convert_dict)

In [ ]:
# Confirm type changes were successful by examining data types again
df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
count_months = df.month.nunique()
count_months

In [ ]:
# 2. How many Martian days' worth of data are there?
count_martian_days = df.sol.nunique()
count_martian_days

In [ ]:
# 3. What is the average low temperature by month?
avg_low_by_month = df.groupby(['month'])['min_temp'].mean()
avg_low_by_month = pd.DataFrame(avg_low_by_month)
avg_low_by_month

In [ ]:
# Plot the average temperature by month
month = df["month"]
min_temp = df["min_temp"]

plt.bar(month, min_temp)
plt.title("Average Min Temp by Month")
plt.xlabel("Month")
plt.ylabel("Temperature")

### On average, Mars' third month has the coldest minimum temperature, and the eighth month is the hottest. However, Mars is always very cold compared to Earth.


In [ ]:
# 4. Average pressure by Martian month
avg_pressure_by_month = df.groupby(['month'])['pressure'].mean()
avg_pressure_by_month = pd.DataFrame(avg_pressure_by_month)
avg_pressure_by_month

In [ ]:
# Plot the average pressure by month
month = df["month"]
pressure = df["pressure"]

plt.bar(month, pressure)
plt.title("Average Pressure by Month")
plt.xlabel("Month")
plt.ylabel("Pressure")

### Average atmospheric pressure is lowest in the sixth month on Mars, and highest in the ninth month.

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
days = df["sol"]
min_temp_daily = df['min_temp']
plt.bar(days, min_temp_daily)
plt.title("Min Temp by Martian Day")
plt.xlabel("Sol")
plt.ylabel("Temperature")

### The distance from peak to peak is about 1500 minus 750, or 750 days. A year on Mars looks to be roughly 750 day, if we are going by the plot above. Google says that there are 687 earth days in a martian year. 

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
df.to_csv('Mars_Data.csv', index=False,header=True)

In [ ]:
browser.quit()